In [1]:
import numpy as np
import keras as K
import tensorflow as tf
import os
import pandas as pd
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np

In [2]:
np.random.seed(1)
print(tf.__version__)

df = pd.read_csv("Gungor_2018_VictorianAuthorAttribution_data-train.csv")

df['author'].unique()

1.14.0


array([ 1,  2,  3,  4,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50])

In [4]:
# p = np.random.permutation(df['author'].unique())
# trainclass = p[:40]
# fewclass = p[40:]

# trainingclasses = df[df['author'].isin(trainclass)]

# fewclasses = df[df['author'].isin(fewclass)]


In [6]:
trainingclasses = df


In [74]:
len(trainingclasses)

53678

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, Model

from keras.layers import *
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional


os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [115]:
seq_len = 64
x = []
y = np.empty((len(trainingclasses) * (1000 // seq_len)), dtype=np.int64)
labels, _ = pd.factorize(trainingclasses['author'])
trainingclasses['author_codes'] = labels
row = 0
for text, author in trainingclasses[['text', 'author_codes']].values:
    words = text.split()
    for j in range(1000//seq_len):
        x.append(" ".join(words[j*seq_len:j*seq_len+seq_len]))
        y[row] = author
        row+=1

x = np.array(x)

In [116]:
# x = np.array(trainingclasses['text'])
# y = np.array(pd.factorize(trainingclasses['author'])[0])

In [119]:
x[2]

'my friend with thanks that i should at once put it in shape for my readers i said i should make a few alterations in it for the sake of dramatic interest but in the main would follow the lines he had given me it would spoil my romance were i to answer on this page the question that must be uppermost in the'

In [120]:
import pickle

max_words = 20000
                
tokenizer = Tokenizer(20000)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

train_x, test_x, train_y, test_y = train_test_split(sequences, y, test_size=0.33, random_state=42)
max_review_len = 64
train_x = K.preprocessing.sequence.pad_sequences(train_x,
  truncating='pre', padding='pre', maxlen=max_review_len)  # pad and chop!
test_x = K.preprocessing.sequence.pad_sequences(test_x,
  truncating='pre', padding='pre', maxlen=max_review_len)

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)
    
word_index = tokenizer.word_index

In [121]:
embeddings_index = {}
f = open(os.path.join('./GLOVE_DIR', 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [122]:
embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [123]:
embedding_layer = Embedding(len(word_index) + 1,
                            50,
                            weights=[embedding_matrix],
                            input_length=max_review_len,
                            trainable=False)

In [124]:
train_x, test_x, train_y, test_y = train_test_split(sequences,
                                                    y, test_size=0.10,
                                                    random_state=42)


In [143]:
input1 = Input(shape=(64,))

x = embedding_layer(input1)

x = Conv1D(128, 3, strides=1)(x)
# x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)

x = Conv1D(128, 3, strides=1)(x)
# x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)

# x = Conv1D(64, 3, strides=1)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.1)(x)

# x = Conv1D(64, 3, strides=1)(x)
# # x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.1)(x)

# x = Conv1D(16, 3, strides=3)(x)
# # x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.1)(x)

feats = Flatten(name='features')(x)

x = Dense(512)(feats)
x = Dropout(0.3)(x)
x = Dense(45)(x)
# x = BatchNormalization()(x)
x = Activation('softmax', name='main_output')(x)

model = Model(inputs=input1, outputs=[x, feats])


print(model.summary()) 

Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 64)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 64, 50)            500050    
_________________________________________________________________
conv1d_75 (Conv1D)           (None, 62, 128)           19328     
_________________________________________________________________
activation_75 (Activation)   (None, 62, 128)           0         
_________________________________________________________________
dropout_76 (Dropout)         (None, 62, 128)           0         
_________________________________________________________________
conv1d_76 (Conv1D)           (None, 60, 128)           49280     
_________________________________________________________________
activation_76 (Activation)   (None, 60, 128)           0  

In [144]:
opt = K.optimizers.Adam(lr=0.001)
model.compile(opt, loss={'main_output': 'sparse_categorical_crossentropy'}, metrics=['acc'])


/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/keras/engine/training_utils.py:819: UserWarning: Output features missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to features.
  'be expecting any data to be passed to {0}.'.format(name))


In [145]:
# 3. train model
bat_size = 512
max_epochs = 50
print("\nStarting training ")
#model.fit(np.array(train_x), train_y, epochs=max_epochs,
#  batch_size=bat_size, shuffle=True, verbose=1) 
model.fit(np.array(train_x), train_y, epochs=max_epochs,
  batch_size=bat_size, shuffle=True, verbose=1, validation_data=(np.array(test_x), test_y)) 
print("Training complete \n")


Starting training 
Train on 724653 samples, validate on 80517 samples
Epoch 1/50
724653/724653 [==============================] - 27s 37us/step - loss: 3.0860 - main_output_loss: 3.0858 - main_output_acc: 0.2136 - val_loss: 2.6842 - val_main_output_loss: 2.6837 - val_main_output_acc: 0.2910
Epoch 2/50
724653/724653 [==============================] - 25s 35us/step - loss: 2.5903 - main_output_loss: 2.5902 - main_output_acc: 0.3177 - val_loss: 2.5006 - val_main_output_loss: 2.5003 - val_main_output_acc: 0.3361
Epoch 3/50
724653/724653 [==============================] - 26s 36us/step - loss: 2.4115 - main_output_loss: 2.4115 - main_output_acc: 0.3596 - val_loss: 2.3477 - val_main_output_loss: 2.3475 - val_main_output_acc: 0.3737
Epoch 4/50
724653/724653 [==============================] - 26s 36us/step - loss: 2.3017 - main_output_loss: 2.3018 - main_output_acc: 0.3854 - val_loss: 2.2527 - val_main_output_loss: 2.2525 - val_main_output_acc: 0.4031
Epoch 5/50
724653/724653 [===============

KeyboardInterrupt: 

In [161]:
evals = sequences[:200*15]
eval_ys = [y[i] for i in range(0,200*15,15)]
predictions, _ = model.predict(np.array(evals))

In [164]:
predictions.reshape((,45))

(3000, 45)

In [ ]:
x2 = np.array(fewclasses['text'])
y2 = np.array(pd.factorize(fewclasses['author'])[0])

In [135]:
from keras.models import load_model

model = load_model('model_fewshot_1.h5')

/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/keras/engine/training_utils.py:819: UserWarning: Output features missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to features.
  'be expecting any data to be passed to {0}.'.format(name))


In [137]:
x3 = []

for i in range(5):
    x3.append(np.random.choice(x2[y2==i]))

In [136]:
print(train_x.shape)
print(x2.shape)
print(x3.shape)

(5, 1000)
(5206,)
(5,)


In [138]:
x3 = np.array(x3)

In [139]:
sequences = tokenizer.texts_to_sequences(x3)
test_x3 = tokenizer.texts_to_sequences(x2)



max_review_len = 1000
train_x = K.preprocessing.sequence.pad_sequences(sequences,
  truncating='pre', padding='pre', maxlen=max_review_len)  # pad and chop!
test_x = K.preprocessing.sequence.pad_sequences(test_x3,
  truncating='pre', padding='pre', maxlen=max_review_len)

In [140]:
train_features, _ = model.predict(train_x)

In [141]:
test_features, _ = model.predict(np.array(test_x3))

In [142]:
train_features = np.expand_dims(train_features, axis=0)
test_features = np.expand_dims(test_features, axis=1)

In [143]:
print(train_features.shape)
print(test_features.shape)

(1, 5, 40)
(5206, 1, 40)


In [144]:
dif = train_features - test_features

In [145]:
print(dif.shape)

(5206, 5, 40)


In [146]:


distance = np.linalg.norm(dif, axis=2)
predictions = np.argmin(distance, axis=1)
'''index = np.argpartition(distance, self.k)
values = index[:self.k]
unique, counts = np.unique(self.y[values], return_counts=True)
prediction = unique[np.argmax(counts)]'''
print(predictions.shape)

(5206,)


In [147]:
np.sum(predictions == y2)/5206

0.19996158278908951

In [50]:
model.save('model_fewshot_1.h5')

In [151]:
min(y2)

0